In [1]:
# Cell 1 — Imports

# Built-in Python library. No installation needed.
# We use it for one job: reading environment variables.
# An environment variable is a value stored outside your code
# (in your .env file) that your program can read safely.
import os

# --- load_dotenv ---
# From the 'python-dotenv' package we installed with uv.
# Its one job: read your .env file and load everything in it
# as environment variables so os.getenv() can find them.
# Without this line, Python has no idea your .env file exists.
from dotenv import load_dotenv

# --- OpenAI ---
# The official OpenAI Python SDK.
# Even though we use OpenRouter, it speaks the same language
# as the OpenAI API — so we use this SDK and point it at
# OpenRouter's address instead. That one change is all it takes.
from openai import OpenAI

print("✅ All imports successful")

✅ All imports successful


In [2]:
# Cell 2 — Load your API key from .env

# load_dotenv() reads your .env file and loads every line into
# memory as an environment variable.
# override=True means: if the variable already exists in the system,
# replace it with the value from OUR .env file.
# This ensures we always use the key from our project, not some
# leftover value from a previous session.
load_dotenv(override=True)

# os.getenv() looks up an environment variable by name and returns its value.
# 'OPENROUTER_API_KEY' must match EXACTLY what you wrote in your .env file —
# same spelling, same capitalisation.
# If it can't find the variable, it returns None instead of crashing.
api_key = os.getenv("OPENROUTER_API_KEY")

# Safety check — if the key is missing, stop here with a clear message.
# 'assert' is a Python keyword that says "this must be true, or stop here."
# The second argument is the error message shown if the check fails.
assert api_key is not None, "❌ OPENROUTER_API_KEY not found. Check your .env file."

# Print only the first 8 characters as confirmation.
# This proves the key loaded without exposing the full key in your output.
# Bots scan GitHub for exposed keys — never print the full key.
print(f"✅ API key loaded. Starts with: {api_key[:8]}...")

✅ API key loaded. Starts with: sk-or-v1...


In [3]:
# Cell 3 — Configure the OpenAI SDK to use OpenRouter

# Create an OpenAI client object.
# A 'client' holds your connection settings and gives you
# methods (functions) to make API calls.
#
# We pass two arguments:
#
# api_key  — the key we loaded from .env.
#             OpenRouter uses this to identify who you are
#             and charge your account for usage.
#
# base_url — the web address the SDK sends requests to.
#             Normally this points to api.openai.com.
#             By changing it to openrouter.ai/api/v1 we redirect
#             ALL calls to OpenRouter instead.
#             This single argument is the entire trick that makes
#             OpenRouter work with the OpenAI SDK.
client = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

print("✅ OpenAI client configured to use OpenRouter")

✅ OpenAI client configured to use OpenRouter


In [4]:
# Cell 4 — Make a test API call

# client.chat.completions.create() sends a message to the AI
# and waits for a response. This is the core function you will
# use constantly throughout.
#
# model      — which AI model to use.
#               'openai/gpt-4o-mini' = GPT-4o Mini via OpenRouter.
#               Fast and cheap — perfect for testing.
#
# messages   — the conversation as a list of message objects.
#               Each message has:
#                 'role'    — 'user' (you) or 'assistant' (the AI)
#                 'content' — what was said
#
# max_tokens — maximum length of the AI's reply.
#               1 token ≈ 0.75 words. 20 tokens ≈ 15 words.
#               We keep it tiny for this test — just need confirmation.
response = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Say exactly this and nothing else: 'OpenRouter connection successful.'"
        }
    ],
    max_tokens=20
)

# The AI's reply text lives at this specific path inside the response:
# response.choices[0].message.content
#
# Breaking it down:
#   .choices      — a list of possible responses (usually just one)
#   [0]           — the first (and usually only) item in that list
#   .message      — the message object inside that response
#   .content      — the actual text string the AI produced
reply = response.choices[0].message.content

print(f"✅ AI responded: {reply}")

✅ AI responded: OpenRouter connection successful.


In [5]:
# Cell 5 — Inspect the full response object

# The response object contains more than just the text.
# This cell shows everything OpenRouter sends back.
# Understanding this structure is useful when debugging.
#
# Things you will see:
#   id                  — unique ID for this specific API call
#   model               — which model actually answered
#   usage               — how many tokens were used (this costs money)
#     prompt_tokens       — tokens in YOUR message to the AI
#     completion_tokens   — tokens in the AI's reply
#     total_tokens        — the sum (what OpenRouter charges you for)
#   choices             — the list of responses the AI gave

print("Full response object:")
print(response)

print("\n--- Token usage breakdown ---")
print(f"Your message used:    {response.usage.prompt_tokens} tokens")
print(f"AI reply used:        {response.usage.completion_tokens} tokens")
print(f"Total tokens charged: {response.usage.total_tokens} tokens")

Full response object:
ChatCompletion(id='gen-1771952511-HsQH4BvmbWIPUuLT0WCc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='OpenRouter connection successful.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1771952511, model='openai/gpt-4o-mini', object='chat.completion', service_tier=None, system_fingerprint='fp_373a14eb6f', usage=CompletionUsage(completion_tokens=5, prompt_tokens=20, total_tokens=25, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0), cost=6e-06, is_byok=False, cost_details={'upstream_inference_cost': 6e-06, 'upstream_inference_prompt_cost': 3e-06, 'upstream_inference_completions_cost': 3e-06}), provider='OpenAI')

--- Token usage b